In [7]:
import eyed3
import os
import mechanicalsoup
import requests
import json
import tagger

brow = mechanicalsoup.StatefulBrowser()
base = "https://api-v2.soundcloud.com"
offset = "?offset=0&limit=100"
add_info="&client_id=tNdzqSQH10kJuLrRhPLbf5wtQEnaXmi1&app_version=1564667264&app_locale=en"
path = "/media/cola/Transfer/Download/SCscrape/"
dlink = "https://sclouddownloader.com/"
d_cmd = 'wget "{}" -O "{}"'


def mk_dir(_path):
    if not os.path.exists(_path):
        os.mkdir(_path)


def download_song(sclink, title, index):
    file_name = path+title+".mp3"
    mess = "downloading: {}".format(file_name)                                    
    if os.path.isfile(file_name):                
        print("SKIP -{}- {}".format(index, mess))
        return index+1
    brow.open(dlink)
    form = brow.select_form('form')
    form["sound-url"] = sclink 
    brow.submit_selected()            
    dpage = brow.get_current_page()    
    aas = dpage.find_all("a")
    for a in aas:            
        try:
            tmp = a["onclick"]
            tmps = tmp.split("'")
            
            file_name = path+title+".mp3"                
            cmd = d_cmd.format(tmps[1], file_name)
            os.system(cmd)   
            tagger.make_tag("")
            print("DONE -{}- {}".format(index, mess)) 
            return index+1
        except Exception as e:            
            # print(e)
            continue
    #print(dpage)
    print("could not download {}".format(sclink))
    return index
            
            
def download_track(element, count):
    username = element["user"]["username"]            
    tr = element["track"]
    if username != tr["user"]["username"]:
        print("SKIP - Repost by user")
        return count+1
    title = tr["title"]
    link = tr["permalink_url"]
    count = download_song(link, title, count+1)                        
    return count
    
            
def download_playlist(element, count):
    username = element["user"]["username"]  
    playlist = element["playlist"]
    if username != playlist["user"]["username"]:
        print("SKIP - Repost by user")
        return count+1
    tracks = playlist["tracks"] 
    for tr in tracks:
        try:
            title = tr["title"]
            link = tr["permalink_url"]
            count = download_song(link, title, count)
        except:
            pass
    return count
          
        
def do_work(req):    
    link = base+req+offset+add_info    
    print("Downloading tracks from {}\n".format(link))
    resp = requests.get(link)
    json_data = json.loads(resp.text)
    collection = json_data["collection"]
    next_ref = json_data["next_href"]
    next_p = next_ref + add_info
    # print(json_data)
    global count
    count = 0 
    while next_ref is not None:        
        next_p = next_ref + add_info

        for el in collection:
            try:
                eltype = el["type"]
                if eltype.lower()=="track":
                    print("downloading track:")
                    count = download_track(el, count)
                elif eltype.lower()=="playlist":
                    print("downloading playlist:")
                    count = download_playlist(el, count)
                else:
                    print("type: "+eltype)
            except Exception as e:
                print(e)
        
        print("\nDone {} .. Scrolling  to {}\n".format(count, next_p))    
        resp = requests.get(next_p)
        json_data = json.loads(resp.text)
        collection = json_data["collection"]        
        next_ref = json_data["next_href"]            
    print("\nDone {}".format(count))    
    

# link to Soundcloud REST API "https://api-v2.soundcloud.com/" 
req = "/stream/users/517620006" # (e.G. 'stream/users/216049')
name = "sven"
path += name+"/"
mk_dir(path)
do_work(req)


downloading track:
SKIP -1- downloading: /media/cola/Transfer/Download/SCscrape/sven/Electrica Salsa feat. Sven Väth (Henrik Schwarz Radio Edit).mp3
downloading track:
SKIP -3- downloading: /media/cola/Transfer/Download/SCscrape/sven/Electrica Salsa feat. Sven Väth (Roman Flügel Remix).mp3
downloading track:
SKIP -5- downloading: /media/cola/Transfer/Download/SCscrape/sven/Electrica Salsa feat. Sven Väth (Henrik Schwarz Dub).mp3
downloading track:
SKIP -7- downloading: /media/cola/Transfer/Download/SCscrape/sven/Electrica Salsa feat. Sven Väth (Henrik Schwarz Remix).mp3
downloading track:
SKIP -9- downloading: /media/cola/Transfer/Download/SCscrape/sven/Ritual Of Life (Adam Port 108 Mix).mp3
downloading track:
SKIP -11- downloading: /media/cola/Transfer/Download/SCscrape/sven/Accident In Paradise (KiNK Remix).mp3
downloading track:
SKIP -13- downloading: /media/cola/Transfer/Download/SCscrape/sven/Robot (Kölsch Remix).mp3
downloading playlist:
SKIP -14- downloading: /media/cola/Transf